In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## Embedding

임베딩 클래스는 텍스트 임베딩 모델과의 인터페이스를 위해 설계된 클래스입니다.

이 클래스는 다양한 임베딩 모델(OpenAI, Cohere, HuggingFace 등)에 대한 표준 인터페이스를 제공하도록 설계되었습니다.

임베딩은 텍스트 조각의 벡터 표현을 생성합니다. 이는 벡터 공간에서 텍스트를 생각하고, 벡터 공간에서 가장 유사한 텍스트 조각을 찾는 시맨틱 검색과 같은 작업을 수행할 수 있다는 점에서 유용합니다.

LangChain의 기본 Embeddings 클래스는 다음의 2가지 형태의 임베딩 함수를 가집니다.

- 문서 임베딩(`embed_documents`): 여러 개의 텍스트를 입력
- 쿼리 임베딩(`embed_query`): 하나의 텍스트를 입력


In [2]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.callbacks import get_openai_callback

openai_embedding = OpenAIEmbeddings()

with get_openai_callback() as cb:
    embedding = openai_embedding.embed_query(
        "안녕하세요? 반갑습니다. 제 이름은 테디입니다."
    )
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost}")
len(embedding)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


호출에 청구된 금액(USD): 	$0.0


1536

In [3]:
embedding = openai_embedding.embed_documents(
    ["안녕하세요?", "반갑습니다", "제 이름은 테디입니다."])
len(embedding), len(embedding[0])

(3, 1536)

In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/appendix-keywords.txt")
docs = loader.load()
len(docs)

1

In [5]:
with get_openai_callback() as cb:
    openai_embedding.embed_query(docs[0].page_content)
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost:.2f}")

호출에 청구된 금액(USD): 	$0.00


In [7]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/상품요약서_신한스포츠&레저보장보험Plus(무배당)_240401_P9.pdf")
pdf = loader.load()
len(pdf)

9

In [8]:
whole_pdf = " ".join([p.page_content for p in pdf])
len(whole_pdf)

6627

In [9]:
with get_openai_callback() as cb:
    openai_embedding.embed_query(whole_pdf)
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost:.2f}")

호출에 청구된 금액(USD): 	$0.00


## HuggingFace Embeddings


In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embedding = HuggingFaceEmbeddings()

text = "안녕하세요? 반갑습니다. 제 이름은 테디입니다."
query_result = hf_embedding.embed_query(text)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
len(query_result)

768

In [3]:
import os
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

# HugginFace 링크: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
hf_embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    model_name="sentence-transformers/all-MiniLM-l6-v2",
)

query_result = hf_embedding.embed_query(text)
len(query_result)

384

허깅페이스의 `BGE` 모델은 최고의 오픈소스 임베딩 모델입니다. BGE 모델은 Beijing Academy of Artificial Intelligence (BAAI) 에서 개발했습니다. BAAI는 AI 연구 및 개발에 종사하는 민간 비영리 단체입니다.


In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
query_result = bge_embedding.embed_query(text)
len(query_result)

384

## CacheBackedEmbeddings


`CacheBackedEmbeddings` 는 임베딩을 저장하거나 임시로 캐싱(caching)하여 다시 계산할 필요가 없도록 할 수 있습니다.

`CacheBackedEmbeddings` 는 임베더를 감싸는 wrapper 클래스로, 임베딩을 key-value 저장소에 캐시합니다.

텍스트는 해시(hash) 되고 해시는 캐시에서 키(key)로 사용됩니다.

`CacheBackedEmbedding` 을 초기화하는 데 주로 지원되는 방법은 `from_bytes_store` 입니다. 여기에는 다음 매개변수가 사용됩니다:

- `underlying_embedder`: 임베딩에 사용할 임베더.
- `document_embedding_cache`: 문서 임베딩을 캐싱하기 위한 바이트 저장소.
- `namespace`: (선택 사항, 기본값은 "") 문서 캐시에 사용할 네임스페이스입니다. 이 네임스페이스는 다른 캐시와의 충돌을 피하기 위해 사용됩니다. 예를 들어 사용된 임베딩 모델의 이름으로 설정합니다.

주의: 서로 다른 임베딩 모델을 사용하여 임베드된 동일한 텍스트의 충돌을 방지하려면 네임스페이스 매개변수를 설정해야 합니다.


In [6]:
from langchain.embeddings import CacheBackedEmbeddings

In [7]:
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

openai_embedding = OpenAIEmbeddings()

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    openai_embedding, store, namespace=openai_embedding.model
)

/home/samuel/Dev/langchain-kr/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# 초기의 캐시는 비어있음
list(store.yield_keys())

[]

In [9]:
cached_embedder.embed_query("안녕하세요? 반갑습니다. 제 이름은 테디입니다.")

[-0.010770357923422545,
 -0.028560170922586725,
 -0.01396275031160441,
 -0.029169455229553432,
 -0.0016501431741946682,
 0.037394783129054976,
 -0.01977633070094087,
 0.0017707305962690542,
 -0.007025802039505753,
 0.01179217717992678,
 0.002200719812573234,
 -0.011988924674710685,
 -0.010941718494472619,
 -0.01339154685589977,
 0.0029988179096187007,
 -0.006340357892660185,
 0.012909197167602226,
 -0.009253495051092532,
 0.011995271638474803,
 -0.01752959866070701,
 0.004978989578953907,
 -0.017580372508174676,
 8.097007036192576e-05,
 0.0015097223052647226,
 0.00012564491144134173,
 0.0007203509923565933,
 0.03599850698030737,
 -0.010922678534502904,
 0.012058738482148066,
 -0.026351516939647025,
 0.019002032786688203,
 0.005315365154273892,
 -0.005981768875488427,
 -0.009678724859480929,
 -0.002898857421285718,
 -0.01600638882461288,
 0.012027005525972752,
 -0.003401833577663348,
 0.01822773487243554,
 0.005699340629518165,
 -0.0031701789463300925,
 -0.015473266220170307,
 0.0011067

## ByteStore 로 교체


다른 바이트 저장소를 사용하려면 `CacheBackedEmbedding` 을 생성할 때 사용하면 됩니다.

아래에서는 동등한 캐시된 임베딩 객체를 생성하지만, 비영구적인 `InMemoryByteStore` 를 대신 사용합니다:


In [10]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

NameError: name 'underlying_embeddings' is not defined

In [11]:
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [12]:
doc_result = embeddings.embed_documents(
    ["Sung is a professor.", "This is another document"]
)
print(doc_result)

[[-0.0038604736328125, -0.028656005859375, 0.0060577392578125, 0.0158233642578125, -0.013580322265625, -0.0175323486328125, 0.006557464599609375, -0.00798797607421875, 0.0010929107666015625, 0.01146697998046875, 0.0189666748046875, 0.02105712890625, -0.0019683837890625, 0.005359649658203125, 0.0038356781005859375, -0.0019855499267578125, 0.00621795654296875, 0.0005564689636230469, 0.0253753662109375, -0.006328582763671875, -0.0208892822265625, 0.0016584396362304688, -0.0011148452758789062, 0.0093231201171875, -0.00383758544921875, 0.0032253265380859375, 0.0106658935546875, 0.01033782958984375, 0.0038051605224609375, 0.0146942138671875, 0.0147247314453125, -0.00943756103515625, 0.0176849365234375, 0.00287628173828125, 0.004291534423828125, -0.007640838623046875, 0.009552001953125, 0.03466796875, -0.0021209716796875, 0.00994873046875, 0.02142333984375, -0.005741119384765625, 0.0005497932434082031, 0.001194000244140625, -0.0033779144287109375, -0.0009002685546875, -0.00244140625, -0.00551

In [13]:
query_result = embeddings.embed_query("What does Sung do?")
print(query_result)

[0.01363372802734375, -0.043426513671875, 0.001598358154296875, 0.01305389404296875, -0.031494140625, -0.01763916015625, 0.0088653564453125, -0.0079193115234375, -0.00745391845703125, 0.01082611083984375, 0.01296234130859375, 0.033172607421875, -0.005279541015625, 0.00591278076171875, -0.016632080078125, -0.01323699951171875, -0.01456451416015625, 0.0150146484375, -0.0157623291015625, -0.007045745849609375, -0.01436614990234375, -0.0014753341674804688, -0.01190185546875, 0.0021839141845703125, -0.021942138671875, -0.00592041015625, 0.01140594482421875, -0.01544952392578125, 0.0169830322265625, 0.006481170654296875, 0.01202392578125, -0.0006933212280273438, 0.01080322265625, 0.004589080810546875, 0.0084228515625, -0.003604888916015625, -0.01114654541015625, 0.0267333984375, 0.0008950233459472656, -0.0081634521484375, 0.016937255859375, -0.005939483642578125, -0.021331787109375, 0.000736236572265625, 0.02606201171875, -0.01334381103515625, -0.00395965576171875, -0.01296234130859375, 0.00